In [1]:
import pandas as pd
import numpy as np
import math
import statistics
from datetime import datetime
import datetime as dt
from datetime import timedelta
import json
import miceforest as mf
from matplotlib import pyplot as plt

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('imputed.csv')

In [21]:
df['icu_stay_days'] = [round(i/86400) for i in df['icu_stay_duration']]

In [29]:
df

,Unnamed: 0,ethnicity,marital_status,language,admission_location,temp_mean,temp_max,temp_min,hr_mean,hr_max,hr_min,spo2_mean,spo2_max,spo2_min,rr_mean,rr_max,rr_min,sbp_mean,sbp_max,sbp_min,dbp_mean,dbp_max,dbp_min,meanbp_mean,meanbp_max,meanbp_min,pt_mean,pt_max,pt_min,ptt_mean,ptt_max,ptt_min,inr_mean,inr_max,inr_min,inr_1_mean,inr_1_max,inr_1_min,fibrinogen_mean,fibrinogen_max,fibrinogen_min,hb_mean,hb_max,hb_min,hematocrit_mean,hematocrit_max,hematocrit_min,wcc_mean,wcc_max,wcc_min,chloride_mean,chloride_max,chloride_min,magnesium_mean,magnesium_max,magnesium_min,potassium_mean,potassium_max,potassium_min,creatinine_mean,creatinine_max,creatinine_min,free_calcium_mean,free_calcium_max,free_calcium_min,sodium_mean,sodium_max,sodium_min,bicarb_mean,bicarb_max,bicarb_min,bun_mean,bun_max,bun_min,hba1c_mean,hba1c_max,hba1c_min,glucose_mean,glucose_max,glucose_min,lactate_mean,lactate_max,lactate_min,po2_mean,po2_max,po2_min,pco2_mean,pco2_max,pco2_min,baseexcess_mean,baseexcess_max,baseexcess_min,ph_mean,ph_max,ph_min,insulin_mean,insulin_max,insulin_min,plt_mean,plt_max,plt_min,gender,insurance,aortic,mit,tricuspid,pulmonary,cabg,weight,height,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_type,hospital_expire_flag,diab_un,diab_cc,duration1,icu_stay_duration,icu_stay_days
0,0,white,SINGLE,ENGL,PHYSICIAN REFERRAL,37.261905,37.944446,36.499998,87.000000,109.0,63.0,97.963636,100.0,92.0,23.862069,43.0,13.0,111.333333,161.0,91.0,59.035088,69.0,43.0,72.622807,86.0,57.000000,12.660000,12.6,12.5,29.800000,29.8,29.8,1.200000,1.2,1.2,1.200000,1.2,1.2,455.0,651.0,277.0,12.200000,13.0,11.4,38.500000,39.0,38.0,12.666667,14.7,10.2,103.400000,107.0,100.0,2.000000,2.0,2.0,3.816667,4.1,3.6,0.833333,0.9,0.8,1.133333,1.15,1.12,138.400000,140.0,136.0,27.666667,32.0,25.0,11.666667,13.0,11.0,6.0,6.0,6.0,122.170213,168.0,87.0,2.600,2.8,2.4,0.750000,2.0,0.0,44.250000,48.0,40.0,0.750000,2.0,0.0,7.385000,7.40,7.36,188.666667,314.000000,0.391930,239.666667,268.0,196.0,M,Private,0,0,0,0,1,84.00,172.72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,CSRU,CSRU,ELECTIVE,0,0,0,5.000000,196721.0,2
1,1,other,DIVORCED,SPAN,PHYSICIAN REFERRAL,37.043305,38.500000,35.200001,90.157407,119.0,76.0,97.339286,100.0,65.0,14.210526,25.0,7.5,99.149533,134.0,75.0,52.696262,75.0,35.0,67.266356,101.0,45.000000,16.775000,19.7,14.2,41.166667,47.5,37.9,1.500000,1.8,1.3,1.500000,1.8,1.3,278.0,277.0,276.0,7.116667,10.0,6.0,21.000000,30.0,18.0,10.100000,10.1,10.1,106.000000,108.0,104.0,1.850000,2.1,1.6,4.200000,4.9,3.4,0.550000,0.6,0.5,1.026250,1.21,0.69,141.333333,143.0,139.0,26.500000,27.0,26.0,13.000000,14.0,12.0,6.1,6.1,6.1,136.300000,195.0,105.0,0.850,0.9,0.8,0.294118,4.0,-3.0,40.764706,50.0,35.0,0.294118,4.0,-3.0,7.401667,7.50,7.35,175.000000,314.000000,0.000000,112.666667,141.0,93.0,F,Self Pay,1,0,0,0,0,60.00,170.18,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,CSRU,CSRU,ELECTIVE,0,0,0,16.483333,196320.0,2
2,2,white,MARRIED,CANT,PHYSICIAN REFERRAL,36.881572,37.799999,35.400002,79.419355,88.0,57.0,98.736842,100.0,93.0,19.112903,39.0,9.0,118.258065,170.0,89.0,52.096774,88.0,22.0,74.623657,121.0,48.666698,23.377778,29.1,17.7,54.750000,59.3,50.2,2.150000,2.7,1.6,2.150000,2.7,1.6,175.0,175.0,175.0,9.600000,11.9,6.5,28.000000,35.0,20.0,17.975000,22.8,9.4,109.000000,113.0,105.0,2.600000,2.6,2.6,4.363636,5.2,3.1,1.333333,1.6,1.2,1.165000,1.32,0.95,140.000000,143.0,139.0,27.333333,29.0,25.0,19.666667,21.0,19.0,6.1,6.1,6.1,124.789474,188.0,92.0,1.200,1.7,0.9,1.571429,4.0,-1.0,40.142857,48.0,33.0,1.571429,4.0,-1.0,7.413333,7.52,7.32,169.333333,294.000000,0.000000,114.250000,148.0,67.0,F,Medicare,1,0,0,0,1,57.00,165.10,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,CSRU,CSRU,ELECTIVE,0,0,0,20.000000,182800.0,2
3,3,white,MARRIED,PTUN,PHYSICIAN REFERRAL,37.532258,38.700001,36.700001,87.939394,104.0,73.0,97.531250,100.0,93.0,16.212121,28.0,10.0,113.191176,162.0,85.0,56.823529,78.0

In [34]:
categorical = ['ethnicity', 
              'marital_status',
              'language',
              'admission_location',
              'gender',
              'insurance',
              'first_careunit',
              'last_careunit',
              'admission_type']
proceduretype=['aortic','mit','tricuspid','pulmonary','cabg']
ptParams = ['weight', 'height']
boolFields = ['reintubation', 'liver_severe', 'liver_mild', 'rheum', 'cvd', 'aids', 'ckd', 'copd', 'arrhythmia', 'pud', 'smoking', 'pvd', 'paraplegia', 
              'ccf', 'met_ca', 't2dm', 't1dm', 'malig', 'mi', 'dementia', 'hospital_expire_flag', 'diab_un', 'diab_cc',]
ptinfo = ['Unnamed: 0']

tsColumns = [i for i in df.columns if '_max' in i or '_min' in i or '_mean' in i]
print([i for i in df.columns if i not in categorical + proceduretype + tsColumns + ptParams + boolFields + ptinfo])

['Unnamed: 0', 'duration1', 'icu_stay_duration', 'icu_stay_days']
